In [1]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import time
from splinter import Browser
import re

In [4]:
def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
    return Browser("chrome", **executable_path, headless=True)


def scrape_info(url):
    browser = init_browser()

    # Visit visitcostarica.herokuapp.com
    browser.visit(url)

    time.sleep(1)
    # Scrape without splinter
    # html = requests.get("https://mars.nasa.gov/news/").text
    # soup = bs.BeautifulSoup(html, 'html.parser')
    
    # Scrape page into Soup
    html = browser.html
    soup = bs(html, "lxml")
    
    browser.quit()
    return soup
    
def scrape_nasa_news():
    url = "https://mars.nasa.gov/news/"
    soup = scrape_info(url)
    news_title = soup.find('div', class_='content_title').text.strip()
    news_p = soup.find('div', class_='article_teaser_body').text.strip()
    return news_title, news_p

def scrape_nasa_spaceimages():
    browser = init_browser()
    browser.visit("https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars")
    time.sleep(1)
    # Click's the link to produce a larger version of the featured image
    browser.click_link_by_partial_text('FULL IMAGE')
    time.sleep(1)
    soup = bs(browser.html, "lxml")
    # finds the relative path to the image, and converts it to a full image path
    img_url = ("https://www.jpl.nasa.gov/spaceimages/" + 
               soup.find('img',class_="fancybox-image")['src'].split("/",2)[-1])
    browser.quit()
    return img_url    

def scrape_twitter():
    url = "https://twitter.com/marswxreport?lang=en"
    soup = scrape_info(url)
    # regular expression pattern to pass to soup's find
    pattern = re.compile(r'InSight sol')
    # find text in the html hat contains the starting message for the weather tweet
    last_tweet = soup.find_all(text=pattern)[0].replace('\n', ' ')
    return last_tweet

def scrape_table():
    html = requests.get("https://space-facts.com/mars/").content
    # the second table has mars's facts
    df = pd.read_html(html)[1]
    return df.to_html()

def scrape_hemisphere():
    wayback = "https://web.archive.org/web/20181114171728/"
    #wayback = ""
    html = requests.get(wayback + 
                        "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars").content
    soup = bs(html, 'lxml')
    items = soup.find_all('div', class_='item')
    #urls = ["https://astrogeology.usgs.gov" + item.find('a')['href'] for item in items]
    urls = ["https://web.archive.org" + item.find('a')['href'] for item in items]
    print(urls)
    hemisphere_image_urls = []
    for path in urls:
        print(path)
        soup_hemisphere = bs(requests.get(path).content, 'lxml')
        title = soup_hemisphere.find('h2', class_='title').text
        img_url = soup_hemisphere.find('div', class_='downloads').find('a')['href']
        hemisphere_image_urls.append({"title": title, "img_url": img_url})
    return hemisphere_image_urls

def scrape():
    news_title, news_p = scrape_nasa_news()
    featured_img_url = scrape_nasa_spaceimages()
    last_tweet = scrape_twitter()
    facts_df = scrape_table()
    hemisphere_img_urls = scrape_hemisphere()
    results = {"news_title": news_title, "news_p": news_p,
     "featured_img_url": featured_img_url, "last_tweet": last_tweet,
     "facts_df": facts_df, "hemisphere_img_urls":hemisphere_img_urls}
    return results

In [5]:
results = scrape()

['https://web.archive.org/web/20181114171728/https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced', 'https://web.archive.org/web/20181114171728/https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced', 'https://web.archive.org/web/20181114171728/https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced', 'https://web.archive.org/web/20181114171728/https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced']
https://web.archive.org/web/20181114171728/https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced
https://web.archive.org/web/20181114171728/https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced
https://web.archive.org/web/20181114171728/https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced
https://web.archive.org/web/20181114171728/https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced


In [7]:
results

dict

In [8]:
from flask import jsonify

In [9]:
jsonify(results)

RuntimeError: Working outside of application context.

This typically means that you attempted to use functionality that needed
to interface with the current application object in some way. To solve
this, set up an application context with app.app_context().  See the
documentation for more information.